In [1]:
# To add a new cell, type '# %%'
# To add a new markdown cell, type '# %% [markdown]'
# %%
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from math import sqrt


 # Đọc dữ liệu và quan sát nhanh dữ liệu California Housing Prices đã được chia thành 3 tập Train, Dev, và Test
 * Lệnh ``pd.read_csv()`` tham khảo tại trang 47 trong sách Hands-on Machine Learning with Scikit-Learn, Keras, and TensorFlow
 * Lệnh ``df.head()`` tham khảo tại trang 47 trong sách Hands-on Machine Learning with Scikit-Learn, Keras, and TensorFlow
 * Lệnh ``df.info()`` tham khảo tại trang 47 trong sách Hands-on Machine Learning with Scikit-Learn, Keras, and TensorFlow

In [2]:
df_train = pd.read_csv('housing_train.csv') # Đọc dữ liệu tập Train (Tập dữ liệu việc huấn luyện mô hình)
df_dev = pd.read_csv('housing_dev.csv')     # Đọc dữ liệu tập Dev   (Tập dữ liệu dành cho việc phát triển/tinh chỉnh mô hình)
df_test = pd.read_csv('housing_test.csv')   # Đọc dữ liệu tập Test  (Tập dữ liệu dành cho việc kiểm tra mô hình)



In [3]:
df_train.info() # Kiểm tra thông tin ban đầu dữ liệu (số dòng, số cột, kiểu dữ liệu của các cột)



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14447 entries, 0 to 14446
Data columns (total 10 columns):
longitude             14447 non-null float64
latitude              14447 non-null float64
housing_median_age    14447 non-null float64
total_rooms           14447 non-null float64
total_bedrooms        14287 non-null float64
population            14447 non-null float64
households            14447 non-null float64
median_income         14447 non-null float64
median_house_value    14447 non-null float64
ocean_proximity       14447 non-null object
dtypes: float64(9), object(1)
memory usage: 1.1+ MB


In [4]:
df_train.head() # Quan sát nhanh 5 dòng dữ liệu đầu tiên



,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-117.83,34.15,20.0,2421.0,306.0,1023.0,298.0,8.0683,451500.0,INLAND
1,-119.85,36.77,27.0,1510.0,344.0,847.0,295.0,2.9315,83200.0,INLAND
2,-122.13,37.67,40.0,1748.0,318.0,914.0,317.0,3.8676,184000.0,NEAR BAY
3,-119.81,34.46,22.0,3488.0,452.0,1479.0,458.0,7.1687,384400.0,NEAR OCEAN
4,-120.47,34.65,32.0,2193.0,430.0,1074.0,377.0,2.3333,130200.0,NEAR OCEAN


In [5]:
df_dev.info() # Kiểm tra thông tin ban đầu dữ liệu (số dòng, số cột, kiểu dữ liệu của các cột)



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2065 entries, 0 to 2064
Data columns (total 10 columns):
longitude             2065 non-null float64
latitude              2065 non-null float64
housing_median_age    2065 non-null float64
total_rooms           2065 non-null float64
total_bedrooms        2048 non-null float64
population            2065 non-null float64
households            2065 non-null float64
median_income         2065 non-null float64
median_house_value    2065 non-null float64
ocean_proximity       2065 non-null object
dtypes: float64(9), object(1)
memory usage: 161.5+ KB


In [6]:
df_dev.head() # Quan sát nhanh 5 dòng dữ liệu đầu tiên



,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-118.36,33.92,26.0,3695.0,1144.0,2308.0,1009.0,2.6667,229300.0,<1H OCEAN
1,-118.41,34.19,45.0,1106.0,225.0,595.0,228.0,3.6625,190700.0,<1H OCEAN
2,-122.48,37.73,38.0,3195.0,828.0,2410.0,778.0,3.1359,350000.0,NEAR OCEAN
3,-117.18,32.76,8.0,3694.0,997.0,1297.0,807.0,3.6492,158900.0,NEAR OCEAN
4,-117.15,33.14,15.0,1070.0,208.0,470.0,217.0,2.3062,158900.0,<1H OCEAN


In [7]:
df_test.info() # Kiểm tra thông tin ban đầu dữ liệu (số dòng, số cột, kiểu dữ liệu của các cột)



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4128 entries, 0 to 4127
Data columns (total 10 columns):
longitude             4128 non-null float64
latitude              4128 non-null float64
housing_median_age    4128 non-null float64
total_rooms           4128 non-null float64
total_bedrooms        4098 non-null float64
population            4128 non-null float64
households            4128 non-null float64
median_income         4128 non-null float64
median_house_value    4128 non-null float64
ocean_proximity       4128 non-null object
dtypes: float64(9), object(1)
memory usage: 322.6+ KB


In [8]:
df_test.head() # Quan sát nhanh 5 dòng dữ liệu đầu tiên


,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.41,37.65,32.0,3436.0,868.0,2583.0,817.0,3.5039,232400.0,NEAR OCEAN
1,-117.91,33.84,25.0,1021.0,252.0,975.0,258.0,3.1250,168100.0,<1H OCEAN
2,-118.35,34.08,52.0,1801.0,313.0,714.0,293.0,4.6838,479000.0,<1H OCEAN
3,-121.02,37.61,33.0,1469.0,370.0,1318.0,349.0,1.7104,59000.0,INLAND
4,-117.65,34.10,30.0,1461.0,341.0,1014.0,345.0,2.4667,106000.0,INLAND


 # Tách thuộc tính "median_house_value" để làm thuộc tính cần dự đoán  cho bài toán hồi quy tuyến tính (Y), các thuộc tính còn lại là dữ kiện (X).
 * Lệnh ``df.drop()`` tham khảo tại trang sách số 63 sách tham khảo Hands-on Machine Learning with Scikit-Learn, Keras, and TensorFlow

In [9]:
train_X = df_train.drop("median_house_value", axis=1)
train_Y = df_train["median_house_value"].copy()
del df_train

dev_X = df_dev.drop("median_house_value", axis=1)
dev_Y = df_dev["median_house_value"].copy()
del df_dev

test_X = df_test.drop("median_house_value", axis=1)
test_Y = df_test["median_house_value"].copy()
del df_test


 # Kiểm tra các thuộc tính bị khuyết giá trị trong tập Train
 * Chúng ta có thể cả các lệnh ``df.isnull()`` hoặc ``df.isna()`` để chỉ giá trị bị khuyết (``True``) / không bị khuyết (``False``)
 * Chúng ta dùng tiếp lệnh ``arr.any(axis=1)`` để trả về các dòng chỉ chứa một cột có giá trị ``True``/``False`` nếu các cột chỉ cần tồn tại một thuộc tính ``True``
 * Chúng ta tính tổng các dòng kết quả trên bằng lệnh ``arr.sum(axis=0)`` sẽ biết được số trường hợp bị khuyết của mỗi thuộc tính

In [10]:
train_X.isnull()



,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity
0,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...
14442,False,False,False,False,False,False,False,False,False
14443,False,False,False,False,False,False,False,False,False
14444,False,False,False,False,False,False,False,False,False
14445,False,False,False,False,False,False,False,False,False


In [11]:
train_X.isnull().any(axis=1)
# train_X.isna().any(axis=1)



0        False
1        False
2        False
3        False
4        False
         ...  
14442    False
14443    False
14444    False
14445    False
14446    False
Length: 14447, dtype: bool

In [12]:
train_X[train_X.isnull().any(axis=1)]
# train_X[train_X.isna().any(axis=1)]



,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity
41,-124.00,40.92,29.0,1429.0,NaN,672.0,266.0,2.9485,<1H OCEAN
204,-120.97,37.43,27.0,1380.0,NaN,810.0,262.0,2.1875,INLAND
403,-118.27,34.07,27.0,1190.0,NaN,1795.0,422.0,1.7016,<1H OCEAN
413,-119.45,36.61,24.0,1302.0,NaN,693.0,243.0,3.7917,INLAND
586,-121.30,38.58,16.0,1537.0,NaN,1125.0,375.0,2.6471,INLAND
...,...,...,...,...,...,...,...,...,...
13865,-121.75,37.11,18.0,3167.0,NaN,1414.0,482.0,6.8773,<1H OCEAN
13918,-118.10,33.74,32.0,2035.0,NaN,934.0,512.0,4.2287,NEAR OCEAN
14004,-122.08,37.37,29.0,1229.0,NaN,707.0,194.0,7.1108,NEAR BAY
14168,-119.38,36.53,38.0,1281.0,NaN,1423.0,293.0,1.9602,INLAND


In [13]:
train_X.isnull().sum(axis = 0)
# train_X.isna().sum(axis = 0)


longitude               0
latitude                0
housing_median_age      0
total_rooms             0
total_bedrooms        160
population              0
households              0
median_income           0
ocean_proximity         0
dtype: int64

 # Xử lý các cột bị khuyết dữ liệu trên tập Train
 1. Phương án 1: Xóa toàn bộ một dòng có cột dữ liệu bị khuyết
 2. Phương án 2: Thay thế giá trị số bằng cách dùng trung bình cộng (``mean``) của toàn cột dữ liệu đó
     * Cách xử lý 1: Dùng lớp ``SimpleImputer`` từ thư viện ``sklearn.impute``
     * Cách xử lý 2: Cách hai thực hiện một số thao tác trên cột

 Các bạn có thể tham khảo tại trang số 63 sách Hands-on Machine Learning with Scikit-Learn, Keras, and TensorFlow

In [14]:
# Cách 1:
imputer_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
imputer_mean.fit(train_X["total_bedrooms"].values.reshape(-1, 1))
tmp_total_bedrooms = imputer_mean.transform(train_X['total_bedrooms'].values.reshape(-1, 1))



In [15]:
# Cách 2:
idx_null = train_X["total_bedrooms"].isnull() # kiểm tra giá trị khuyết
mean_total_bedrooms = train_X["total_bedrooms"][train_X["total_bedrooms"].isna() == False].mean() # tính trung bình cộng các giá trị không bị khuyết
train_X["total_bedrooms"].fillna(mean_total_bedrooms, inplace=True) # thay thế các dòng không bị khuyết bởi giá trị trung bình
print(train_X["total_bedrooms"][idx_null == True]) # in ra màn hình giá trị các dòng bị khuyết ban đầu để kiểm tra



41       538.66578
204      538.66578
403      538.66578
413      538.66578
586      538.66578
           ...    
13865    538.66578
13918    538.66578
14004    538.66578
14168    538.66578
14371    538.66578
Name: total_bedrooms, Length: 160, dtype: float64


In [16]:
print((train_X["total_bedrooms"] == tmp_total_bedrooms.squeeze()).all()) # Kiểm tra kết quả Cách 1 và Cách 2 có giống nhau hay không?

print((train_X["total_bedrooms"].isna() == False).all()) # Kiểm tra kết quả sau khi áp dụng Cách 2 còn giá trị nào bị khuyết hay không?


True
True


 # Yêu cầu 1: Xử lý các cột bị khuyết dữ liệu trên tập Dev/Test

 ## Xử lí dữ liệu bị khuyết trên tập Dev
 ### 1. Kiểm tra các thuộc tính bị khuyết giá trị

In [17]:
dev_X.isnull()


,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity
0,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...
2060,False,False,False,False,False,False,False,False,False
2061,False,False,False,False,False,False,False,False,False
2062,False,False,False,False,False,False,False,False,False
2063,False,False,False,False,False,False,False,False,False


 ### 2. Tìm các giá trị bị khuyết

In [18]:
dev_X_Null_Index = dev_X.isnull().any(axis=1)
dev_X[dev_X_Null_Index]


,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity
91,-122.48,37.70,33.0,4492.0,NaN,3477.0,1537.0,3.0546,NEAR OCEAN
234,-122.38,37.73,40.0,543.0,NaN,259.0,89.0,2.2167,NEAR BAY
264,-118.16,34.01,40.0,1552.0,NaN,1919.0,427.0,2.2596,<1H OCEAN
365,-119.30,36.57,32.0,728.0,NaN,461.0,149.0,3.0156,INLAND
405,-117.64,33.45,26.0,1528.0,NaN,607.0,218.0,6.2871,NEAR OCEAN
734,-117.09,32.77,31.0,3062.0,NaN,1263.0,539.0,3.0875,NEAR OCEAN
830,-118.25,33.94,43.0,793.0,NaN,736.0,231.0,0.8527,<1H OCEAN
868,-118.18,34.19,48.0,1371.0,NaN,528.0,155.0,15.0001,<1H OCEAN
1246,-119.78,36.82,25.0,5016.0,NaN,2133.0,928.0,3.6250,INLAND
1290,-122.01,37.94,23.0,3741.0,NaN,1339.0,499.0,6.7061,NEAR BAY


In [19]:
dev_X.isnull().sum(axis = 0) # Tính tổng các giá trị bị khuyết


longitude              0
latitude               0
housing_median_age     0
total_rooms            0
total_bedrooms        17
population             0
households             0
median_income          0
ocean_proximity        0
dtype: int64

 ### 3. Xử lí các cột bị khuyết dữ liệu
 * Lọc tất cả các phần tử bị khuyết và thay thế các phần tử đó bằng giá trị trung bình của cột đó

In [20]:
dev_X_imputer_mean = SimpleImputer() # Default SimpleImputer(missing_values=np.nan, strategy="mean", fill_value=None, verbose=0, copy=True, add_indicator=False)
dev_X_imputer_mean.fit(dev_X["total_bedrooms"].values.reshape(-1,1))
dev_X["total_bedrooms"] = dev_X_imputer_mean.transform(dev_X["total_bedrooms"].values.reshape(-1,1))


 ## Xử lí dữ liệu bị khuyết trên tập Test

In [21]:
test_X_Null_Index = test_X.isnull().any(axis=1)
test_X[test_X_Null_Index]


,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity
440,-118.12,34.08,35.0,2248.0,NaN,1762.0,622.0,3.0000,<1H OCEAN
725,-121.77,39.66,20.0,3759.0,NaN,1705.0,600.0,4.7120,INLAND
815,-118.31,34.09,37.0,773.0,NaN,835.0,312.0,1.8576,<1H OCEAN
849,-118.59,34.20,21.0,1789.0,NaN,2300.0,677.0,2.7540,<1H OCEAN
860,-119.46,35.14,30.0,2943.0,NaN,1565.0,584.0,2.5313,INLAND
920,-122.70,38.35,14.0,2313.0,NaN,954.0,397.0,3.7813,<1H OCEAN
1116,-118.02,33.89,36.0,1375.0,NaN,670.0,221.0,5.0839,<1H OCEAN
1124,-117.66,33.51,18.0,2626.0,NaN,1302.0,522.0,4.0167,<1H OCEAN
1453,-118.45,34.30,27.0,2676.0,NaN,2661.0,623.0,4.3047,<1H OCEAN
1480,-122.65,38.25,23.0,4030.0,NaN,1852.0,778.0,3.4020,<1H OCEAN


In [22]:
test_X.isnull().sum(axis = 0)


longitude              0
latitude               0
housing_median_age     0
total_rooms            0
total_bedrooms        30
population             0
households             0
median_income          0
ocean_proximity        0
dtype: int64

In [23]:
test_X_imputer_mean = SimpleImputer() # Default SimpleImputer(missing_values=np.nan, strategy="mean", fill_value=None, verbose=0, copy=True, add_indicator=False)
test_X_imputer_mean.fit(test_X["total_bedrooms"].values.reshape(-1,1))
test_X["total_bedrooms"] = test_X_imputer_mean.transform(test_X["total_bedrooms"].values.reshape(-1,1))

 # Yêu cầu 2: Thực hiện các thí nghiệm với việc thêm các thuộc tính tích lũy dẫn
 Chúng ta có danh sách các thuộc tính: ``longitude``, ``latitude``, ``housing_median_age``, ``total_rooms``, ``total_bedrooms``, ``population``, ``households``, ``median_income``, ``ocean_proximity``

 1. Thí nghiệm 1: ``longitude``
 2. Thí nghiệm 2: ``longitude``, ``latitude``
 3. Thí nghiêm 3: ``longitude``, ``latitude``, ``housing_median_age``
 4. Thí nghiệm 4: v.v...

 Lưu ý: Trong quá trình làm, các bạn sẽ tham khảo cách xử lý thuộc tính ``ocean_proximity`` bằng lớp ``LabelEncoder`` từ thư viện ``sklearn.preprocessing``

In [24]:
features_list_1 = ["longitude"]
model_1 = sm.OLS(train_Y, train_X[features_list_1]).fit()
rmse_train_1 = sqrt(mean_squared_error(train_Y, model_1.predict(train_X[features_list_1])))
# If True returns MSE value, if False returns RMSE value.
rmse_dev_1 = sqrt(mean_squared_error(dev_Y, model_1.predict(dev_X[features_list_1])))
rmse_test_1 = sqrt(mean_squared_error(test_Y, model_1.predict(test_X[features_list_1])))



In [25]:
features_list_2 = ["longitude", "latitude"]
model_2 = sm.OLS(train_Y, train_X[features_list_2]).fit()
rmse_train_2 = sqrt(mean_squared_error(train_Y, model_2.predict(train_X[features_list_2])))
rmse_dev_2 = sqrt(mean_squared_error(dev_Y, model_2.predict(dev_X[features_list_2])))
rmse_test_2 = sqrt(mean_squared_error(test_Y, model_2.predict(test_X[features_list_2])))


In [26]:
features_list_3 = ["longitude", "latitude", "housing_median_age"]
model_3 = sm.OLS(train_Y, train_X[features_list_3]).fit()
rmse_train_3 = sqrt(mean_squared_error(train_Y, model_3.predict(train_X[features_list_3])))
rmse_dev_3 = sqrt(mean_squared_error(dev_Y, model_3.predict(dev_X[features_list_3])))
rmse_test_3 = sqrt(mean_squared_error(test_Y, model_3.predict(test_X[features_list_3])))


In [27]:
features_list_4 = ["longitude", "latitude", "housing_median_age", "total_rooms"]
model_4 = sm.OLS(train_Y, train_X[features_list_4]).fit()
rmse_train_4 = sqrt(mean_squared_error(train_Y, model_4.predict(train_X[features_list_4])))
rmse_dev_4 = sqrt(mean_squared_error(dev_Y, model_4.predict(dev_X[features_list_4])))
rmse_test_4 = sqrt(mean_squared_error(test_Y, model_4.predict(test_X[features_list_4])))


In [28]:
features_list_5 = ["longitude", "latitude", "housing_median_age", "total_rooms", "total_bedrooms"]
model_5 = sm.OLS(train_Y, train_X[features_list_5]).fit()
rmse_train_5 = sqrt(mean_squared_error(train_Y, model_5.predict(train_X[features_list_5])))
rmse_dev_5 = sqrt(mean_squared_error(dev_Y, model_5.predict(dev_X[features_list_5])))
rmse_test_5 = sqrt(mean_squared_error(test_Y, model_5.predict(test_X[features_list_5])))



In [29]:
features_list_6 = ["longitude", "latitude", "housing_median_age", "total_rooms", "total_bedrooms", "population"]
model_6 = sm.OLS(train_Y, train_X[features_list_6]).fit()
rmse_train_6 = sqrt(mean_squared_error(train_Y, model_6.predict(train_X[features_list_6])))
rmse_dev_6 = sqrt(mean_squared_error(dev_Y, model_6.predict(dev_X[features_list_6])))
rmse_test_6 = sqrt(mean_squared_error(test_Y, model_6.predict(test_X[features_list_6])))


In [30]:
features_list_7 = ["longitude", "latitude", "housing_median_age", "total_rooms", "total_bedrooms", "population", "households"]
model_7 = sm.OLS(train_Y, train_X[features_list_7]).fit()
rmse_train_7 = sqrt(mean_squared_error(train_Y, model_7.predict(train_X[features_list_7])))
rmse_dev_7 = sqrt(mean_squared_error(dev_Y, model_7.predict(dev_X[features_list_7])))
rmse_test_7 = sqrt(mean_squared_error(test_Y, model_7.predict(test_X[features_list_7])))


In [31]:
features_list_8 = ["longitude", "latitude", "housing_median_age", "total_rooms", "total_bedrooms", "population", "households", "median_income"]
model_8 = sm.OLS(train_Y, train_X[features_list_8]).fit()
rmse_train_8 = sqrt(mean_squared_error(train_Y, model_8.predict(train_X[features_list_8])))
rmse_dev_8 = sqrt(mean_squared_error(dev_Y, model_8.predict(dev_X[features_list_8])))
rmse_test_8 = sqrt(mean_squared_error(test_Y, model_8.predict(test_X[features_list_8])))


 Ta thấy feature `ocean_proximity` thuộc dạng phân loại (categorical variables), nhưng vấn đề là `sm.OLS` không thể xử lí được loại dữ liệu
 này. Nhưng chúng ta có thể sử dụng `sklearn.reprocessing.LabelEncoder` để chuyển các giá trị phân loại (categorical variables)
 thành các biến giả (dummy/indicator variables)

 * Tìm những phần tử trong feature `ocean_proximity`

In [32]:
train_X.pivot_table(index =['ocean_proximity'], aggfunc='size')


ocean_proximity
<1H OCEAN     6432
INLAND        4557
ISLAND           4
NEAR BAY      1591
NEAR OCEAN    1863
dtype: int64

 * Chuyển chúng thành biến giả

In [33]:
LE_Ocean = LabelEncoder()
LE_Ocean.fit(['<1H OCEAN', 'INLAND', 'ISLAND', 'NEAR BAY', 'NEAR OCEAN'])
train_X['ocean_proximity'] = LE_Ocean.transform(train_X["ocean_proximity"].values.reshape(-1,1))
test_X['ocean_proximity'] = LE_Ocean.transform(test_X["ocean_proximity"].values.reshape(-1,1))
dev_X['ocean_proximity'] = LE_Ocean.transform(dev_X["ocean_proximity"].values.reshape(-1,1))


In [34]:
train_X['ocean_proximity'].head()


0    1
1    1
2    3
3    4
4    4
Name: ocean_proximity, dtype: int32

In [35]:
test_X['ocean_proximity'].head()


0    4
1    0
2    0
3    1
4    1
Name: ocean_proximity, dtype: int32

In [36]:
dev_X['ocean_proximity'].head()


0    0
1    0
2    4
3    4
4    0
Name: ocean_proximity, dtype: int32

In [37]:
features_list_9 = ["longitude", "latitude", "housing_median_age", "total_rooms", "total_bedrooms", "population", "households", "median_income", "ocean_proximity"]
model_9 = sm.OLS(train_Y, train_X[features_list_9]).fit()
rmse_train_9 = sqrt(mean_squared_error(train_Y, model_9.predict(train_X[features_list_9])))
rmse_dev_9 = sqrt(mean_squared_error(dev_Y, model_9.predict(dev_X[features_list_9])))
rmse_test_9 = sqrt(mean_squared_error(test_Y, model_9.predict(test_X[features_list_9])))


 # Yêu cầu 3: Trình bày kết quả mô hình vào một bảng bằng thư viện Pandas

In [38]:
df_result = pd.DataFrame(data = {'RMSE_Train': [rmse_train_1, rmse_train_2, rmse_train_3, rmse_train_4, rmse_train_5, rmse_train_6, rmse_train_7, rmse_train_8, rmse_train_9],
                                 'RMSE_Dev': [rmse_dev_1, rmse_dev_2, rmse_dev_3, rmse_dev_4, rmse_dev_5, rmse_dev_6, rmse_dev_7 ,rmse_dev_8, rmse_dev_9],
                                 'RMSE_Test': [rmse_test_1, rmse_test_2, rmse_test_3, rmse_test_4, rmse_test_5, rmse_test_6, rmse_test_7, rmse_test_8, rmse_test_9]},
                         index = ['longitude',
                                  'longitude + latitude',
                                  'longitude + latitude + housing_median_age',
                                  'longitude + latitude + housing_median_age + total_rooms',
                                  'longitude + latitude + housing_median_age + total_rooms + total_bedrooms',
                                  'longitude + latitude + housing_median_age + total_rooms + total_bedrooms + population',
                                  'longitude + latitude + housing_median_age + total_rooms + total_bedrooms + population + households',
                                  'longitude + latitude + housing_median_age + total_rooms + total_bedrooms + population + households + median_income',
                                  'longitude + latitude + housing_median_age + total_rooms + total_bedrooms + population + households + median_income + ocean_proximity'
                                  ])

display(df_result.round(3))


,RMSE_Train,RMSE_Dev,RMSE_Test
longitude,114808.126,114556.560,117298.758
longitude + latitude,111492.090,112236.757,113827.915
longitude + latitude + housing_median_age,111139.090,111531.578,113134.299
longitude + latitude + housing_median_age + total_rooms,109475.782,110283.774,111129.497
longitude + latitude + housing_median_age + total_rooms + total_bedrooms,106304.615,107283.688,107815.117
longitude + latitude + housing_median_age + total_rooms + total_bedrooms + population,102680.519,104604.796,103410.173
longitude + latitude + housing_median_age + total_rooms + total_bedrooms + population + households,101519.985,102242.234,102033.380
longitude + latitude + housing_median_age + total_rooms + total_bedrooms + population + households + median_income,74885.866,75358.064,75006.686
longitude + latitude + housing_median_age + total_rooms + total_bedrooms + population + households + median_income + ocean_proximity,74421.501,74799.193,74619.996


 # Yêu cầu 4: Nhận xét các kết quả trên
 Ta thấy giá trị hiệu suất ``RMSE`` giảm khi có thêm feature mới. Model hoạt động càng chính xác hơn.

 # Yêu cầu 5: Thực hiện lại các Yêu cầu 2, 3, 4 khi dùng mô hình ``Support Vector Machine regressor`` (``sklearn.svm.SVR``) để giải quyết bài toán hồi quy tuyến tính
 * Import thư viện

In [39]:
from sklearn.svm import SVR
regressor = SVR(kernel='rbf')


 * Áp dụng SVM lên thuộc tính ``longitude``

In [40]:
SVM_model_1 = regressor.fit(train_X[features_list_1].values.reshape(-1,1), train_Y)
SVM_rmse_train_1 = sqrt(mean_squared_error(train_Y, SVM_model_1.predict(train_X[features_list_1])))
SVM_rmse_dev_1 = sqrt(mean_squared_error(dev_Y, SVM_model_1.predict(dev_X[features_list_1])))
SVM_rmse_test_1 = sqrt(mean_squared_error(test_Y, SVM_model_1.predict(test_X[features_list_1])))


 * Tương tự áp dụng lên nhiều thuộc tính khác

In [41]:
SVM_model_2 = regressor.fit(train_X[features_list_2].values.reshape(-1,2), train_Y)
SVM_rmse_train_2 = sqrt(mean_squared_error(train_Y, SVM_model_2.predict(train_X[features_list_2])))
SVM_rmse_dev_2 = sqrt(mean_squared_error(dev_Y, SVM_model_2.predict(dev_X[features_list_2])))
SVM_rmse_test_2 = sqrt(mean_squared_error(test_Y, SVM_model_2.predict(test_X[features_list_2])))


In [42]:
SVM_model_3 = regressor.fit(train_X[features_list_3].values.reshape(-1,3), train_Y)
SVM_rmse_train_3 = sqrt(mean_squared_error(train_Y, SVM_model_3.predict(train_X[features_list_3])))
SVM_rmse_dev_3 = sqrt(mean_squared_error(dev_Y, SVM_model_3.predict(dev_X[features_list_3])))
SVM_rmse_test_3 = sqrt(mean_squared_error(test_Y, SVM_model_3.predict(test_X[features_list_3])))


In [43]:
SVM_model_4 = regressor.fit(train_X[features_list_4].values.reshape(-1,4), train_Y)
SVM_rmse_train_4 = sqrt(mean_squared_error(train_Y, SVM_model_4.predict(train_X[features_list_4])))
SVM_rmse_dev_4 = sqrt(mean_squared_error(dev_Y, SVM_model_4.predict(dev_X[features_list_4])))
SVM_rmse_test_4 = sqrt(mean_squared_error(test_Y, SVM_model_4.predict(test_X[features_list_4])))


In [44]:
SVM_model_5 = regressor.fit(train_X[features_list_5].values.reshape(-1,5), train_Y)
SVM_rmse_train_5 = sqrt(mean_squared_error(train_Y, SVM_model_5.predict(train_X[features_list_5])))
SVM_rmse_dev_5 = sqrt(mean_squared_error(dev_Y, SVM_model_5.predict(dev_X[features_list_5])))
SVM_rmse_test_5 = sqrt(mean_squared_error(test_Y, SVM_model_5.predict(test_X[features_list_5])))


In [45]:
SVM_model_6 = regressor.fit(train_X[features_list_6].values.reshape(-1,6), train_Y)
SVM_rmse_train_6 = sqrt(mean_squared_error(train_Y, SVM_model_6.predict(train_X[features_list_6])))
SVM_rmse_dev_6 = sqrt(mean_squared_error(dev_Y, SVM_model_6.predict(dev_X[features_list_6])))
SVM_rmse_test_6 = sqrt(mean_squared_error(test_Y, SVM_model_6.predict(test_X[features_list_6])))


In [46]:
SVM_model_7 = regressor.fit(train_X[features_list_7].values.reshape(-1,7), train_Y)
SVM_rmse_train_7 = sqrt(mean_squared_error(train_Y, SVM_model_7.predict(train_X[features_list_7])))
SVM_rmse_dev_7 = sqrt(mean_squared_error(dev_Y, SVM_model_7.predict(dev_X[features_list_7])))
SVM_rmse_test_7 = sqrt(mean_squared_error(test_Y, SVM_model_7.predict(test_X[features_list_7])))


In [47]:
SVM_model_8 = regressor.fit(train_X[features_list_8].values.reshape(-1,8), train_Y)
SVM_rmse_train_8 = sqrt(mean_squared_error(train_Y, SVM_model_8.predict(train_X[features_list_8])))
SVM_rmse_dev_8 = sqrt(mean_squared_error(dev_Y, SVM_model_8.predict(dev_X[features_list_8])))
SVM_rmse_test_8 = sqrt(mean_squared_error(test_Y, SVM_model_8.predict(test_X[features_list_8])))


In [48]:
SVM_model_9 = regressor.fit(train_X[features_list_9].values.reshape(-1,9), train_Y)
SVM_rmse_train_9 = sqrt(mean_squared_error(train_Y, SVM_model_9.predict(train_X[features_list_9])))
SVM_rmse_dev_9 = sqrt(mean_squared_error(dev_Y, SVM_model_9.predict(dev_X[features_list_9])))
SVM_rmse_test_9 = sqrt(mean_squared_error(test_Y, SVM_model_9.predict(test_X[features_list_9])))


 ## Trình bày kết quả RMSE khi dùng SVM

In [49]:
df_result = pd.DataFrame(data = {'RMSE_Train': [SVM_rmse_train_1, SVM_rmse_train_2, SVM_rmse_train_3, SVM_rmse_train_4, SVM_rmse_train_5, SVM_rmse_train_6, SVM_rmse_train_7, SVM_rmse_train_8, SVM_rmse_train_9],
                                 'RMSE_Dev': [SVM_rmse_dev_1, SVM_rmse_dev_2, SVM_rmse_dev_3, SVM_rmse_dev_4, SVM_rmse_dev_5, SVM_rmse_dev_6, SVM_rmse_dev_7, SVM_rmse_dev_8, SVM_rmse_dev_9],
                                 'RMSE_Test': [SVM_rmse_test_1, SVM_rmse_test_2, SVM_rmse_test_3, SVM_rmse_test_4, SVM_rmse_test_5, SVM_rmse_test_6, SVM_rmse_test_7, SVM_rmse_test_8, SVM_rmse_test_9]},
                         index = ['longitude',
                                  'longitude + latitude',
                                  'longitude + latitude + housing_median_age',
                                  'longitude + latitude + housing_median_age + total_rooms',
                                  'longitude + latitude + housing_median_age + total_rooms + total_bedrooms',
                                  'longitude + latitude + housing_median_age + total_rooms + total_bedrooms + population',
                                  'longitude + latitude + housing_median_age + total_rooms + total_bedrooms + population + households',
                                  'longitude + latitude + housing_median_age + total_rooms + total_bedrooms + population + households + median_income',
                                  'longitude + latitude + housing_median_age + total_rooms + total_bedrooms + population + households + median_income + ocean_proximity'
                                  ])

display(df_result.round(3))

,RMSE_Train,RMSE_Dev,RMSE_Test
longitude,117877.183,117493.850,120505.370
longitude + latitude,117807.608,117425.087,120435.215
longitude + latitude + housing_median_age,118043.892,117669.980,120667.953
longitude + latitude + housing_median_age + total_rooms,118075.070,117699.874,120700.695
longitude + latitude + housing_median_age + total_rooms + total_bedrooms,118075.160,117699.967,120700.786
longitude + latitude + housing_median_age + total_rooms + total_bedrooms + population,118075.152,117699.960,120700.778
longitude + latitude + housing_median_age + total_rooms + total_bedrooms + population + households,118075.152,117699.960,120700.778
longitude + latitude + housing_median_age + total_rooms + total_bedrooms + population + households + median_income,118075.152,117699.960,120700.778
longitude + latitude + housing_median_age + total_rooms + total_bedrooms + population + households + median_income + ocean_proximity,118075.152,117699.960,120700.778


 # Yêu cầu 4: Nhận xét các kết quả trên
 khi dùng mô hình ``Support Vector Machine regressor`` thì ta thấy mô hình lúc train nhiều feature thì cực kì lâu và lại cho ra kết quả
 không như mong đợi (chỉ số RMSE càng tăng khi ta cho thêm nhiều feature vào)